# Submit InSAR jobs for new granules

Install dependencies:

In [ ]:
!pip install asf-search hyp3-sdk

import asf_search
from hyp3_sdk import HyP3

Define search parameters and job specification:

In [ ]:
search_parameters = {
    "start": "2023-07-20T00:00:00Z",
    "end": "2023-07-25T00:00:00Z",
    "intersectsWith":
        "POLYGON((-110.7759 44.8543,-101.3998 44.8543,-101.3998 50.8183,-110.7759 50.8183,-110.7759 44.8543))",
    "platform": "S1",
    "processingLevel": "SLC",
}
job_specification = {
    "job_parameters": {},
    "job_type": "INSAR_GAMMA",
    "name": "new-insar-job"
}

Construct a list of unprocessed granules:

In [ ]:
hyp3 = HyP3(api_url='https://hyp3-test-api.asf.alaska.edu/')

previous_jobs = hyp3.find_jobs(
    name=job_specification['name'],
    job_type=job_specification['job_type'],
)
processed_granules = [job.job_parameters['granules'][0] for job in previous_jobs]

search_results = asf_search.search(**search_parameters)
search_results.raise_if_incomplete()

unprocessed_granules = [
    result for result in search_results if result.properties['sceneName'] not in processed_granules
]

Submit a new job for each unprocessed granule:

In [ ]:
from copy import deepcopy

def get_neighbors(granule: asf_search.ASFProduct, platform: str, depth=2) -> list[str]:
    stack = asf_search.baseline_search.stack_from_product(granule)
    stack.raise_if_incomplete()
    stack = [item for item in stack if
             item.properties['temporalBaseline'] < 0 and item.properties['sceneName'].startswith(platform)]
    neighbors = [item.properties['sceneName'] for item in stack[-depth:]]
    return neighbors


def get_jobs_for_granule(granule: asf_search.ASFProduct) -> list[dict]:
    jobs = []
    neighbors = get_neighbors(granule, search_parameters['platform'])

    for neighbor in neighbors:
        job = deepcopy(job_specification)
        job['job_parameters']['granules'] = [granule.properties['sceneName'], neighbor]
        jobs.append(job)

    return jobs


new_jobs = [job for granule in unprocessed_granules for job in get_jobs_for_granule(granule)]

print(f'Submitting {len(new_jobs)} jobs')
if new_jobs:
    hyp3.submit_prepared_jobs(new_jobs)